In [1]:
import pandas as pd
import os
import numpy as np
import re

In [2]:
def add_space_after_punctuation(text):
    """Adds a single space after each period and comma in the given text."""
    text = re.sub(r'(?<=[.,])(?=[^\s])', ' ', text)
    return text

def replace_terms(text, search_terms, replacement_terms):
    synonym_map = {
        syn.lower(): rep for syn, rep in zip(search_terms, replacement_terms)
    }
    sorted_terms = sorted(synonym_map.items(), key=lambda x: len(x[0]), reverse=True)
    pattern = re.compile('|'.join(re.escape(term) for term, _ in sorted_terms), flags=re.IGNORECASE)

    result = []
    last_index = 0
    replaced_spans = []

    for match in pattern.finditer(text):
        start, end = match.span()
        if any(rs < end and start < re for rs, re in replaced_spans):
            continue

        matched_text = match.group(0)
        matched_text_lower = matched_text.lower()

        if matched_text_lower in synonym_map:
            filename = synonym_map[matched_text_lower]
            replacement = f"[{matched_text}]({filename}.md)"
        else:
            replacement = matched_text

        result.append(text[last_index:start])
        result.append(replacement)
        replaced_spans.append((start, end))
        last_index = end

    result.append(text[last_index:])
    return ''.join(result)

def bold_first_sentence(text):
    first_period_index = text.find('.')
    if first_period_index != -1:
        return f"**{text[:first_period_index + 1]}**{text[first_period_index + 1:]}"
    else:
        # If there's no period, bold the entire block
        return f"**{text}**"

def bold_until_colon(text):
    first_colon_index = text.find(':')
    if first_colon_index != -1:
        return f"**{text[:first_colon_index + 1]}**{text[first_colon_index + 1:]}"
    else:
        # If there's no colon, bold the entire block
        return f"**{text}**"

def check_for_letters(text):
    return any(char.isalpha() for char in text)

def find_character_after_index(text, char, start_index):
    """
    Finds the first instance of a character in a string after a certain index.

    Args:
        text (str): The string to search in.
        char (str): The character to find.
        start_index (int): The index to start searching from.

    Returns:
        int: The index of the first instance of the character after the start index, or -1 if not found.
    """
    if start_index < 0 or start_index >= len(text):
        return -1

    for i in range(start_index, len(text)):
        if text[i] == char:
            return i
    return -1

def generate_mkdocs_nav(folder_path, output_file="mkdocs_nav.txt", header_name=None):
    if not os.path.isdir(folder_path):
        raise ValueError(f"Folder does not exist: {folder_path}")

    md_files = [f for f in os.listdir(folder_path) if f.endswith(".md")]

    # Prepare (display_name, filename) tuples
    entries = []
    for filename in md_files:
        name_without_ext = os.path.splitext(filename)[0]
        display_name = name_without_ext.replace("_", " ").capitalize()
        entries.append((display_name, filename))

    # Sort by display_name
    entries.sort(key=lambda x: x[0].lower())

    # Generate nav lines
    nav_lines = []
    if header_name:
        nav_lines.append(f"  - {header_name}")
    indent = "    " if header_name else ""

    for display_name, filename in entries:
        nav_lines.append(f"{indent}- {display_name}: {filename}")

    with open(output_file, "w", encoding="utf-8") as f:
        f.write("\n".join(nav_lines))

    print(f"Nav file written to: {output_file}")

# Fix redundant official names

In [3]:
# get plants with illustrations
images = pd.read_csv("./2_illustrations/assets.csv")

# get references to words in text
texts = pd.read_csv("./1_parse_text/verified_synonym_matches.csv")


In [4]:
#“Iztac oco-xochitl.md" in plants without was already in plants with 
#(ID049, p007_02 in with pics, has a pic) 
#(ID038, many references, no pics)
#ACTION: change ID049 to ID038 in both images and texts
images['ID'] = images['ID'].replace("ID038", "ID049")
texts['ID'] = texts['ID'].replace("ID038", "ID049")

In [5]:
#“Tonatiuh yxiuh.md” in plants without was already in plants with 
#(ID184, p012_01 in with pics, has a pic, only pic no text) official: Xiuhuitl tonalco-mochiva hahuachcho
#(ID257, p042_01 in with pics) official: Tonatiuh yxiuh
#(ID258, p051_02 in with pics, has a pic) official: Tonatiuh yxiuh
#ACTION: ID257 is Tonatiuh yxiuh v1, in both images and text
#ACTION: ID258 is Tonatiuh yxiuh v2, in both images and text

images.loc[images['ID'] == 'ID257', 'official_name'] = "Tonatiuh yxiuh v1"
texts.loc[texts['ID'] == 'ID257', 'official_name'] = "Tonatiuh yxiuh v1"

images.loc[images['ID'] == 'ID258', 'official_name'] = "Tonatiuh yxiuh v2"
texts.loc[texts['ID'] == 'ID258', 'official_name'] = "Tonatiuh yxiuh v2"

In [6]:
#“a-camallo-tetl.md” in stones was already in plants with (ID265) 
#(ID265, p113_01, in with pics, has a picture) (IT'S A STONE WITH A PIC)
#(ID279, multiple references, is a stone)
#ACTION: Capitalize official name of ID265, change to A-camallo-tetl v1, in both images and text
#ACTION: lowercase ID279 a-camallo-tetl v2, should only be in text

images.loc[images['ID'] == 'ID265', 'official_name'] = "A-camallo-tetl v1"
texts.loc[texts['ID'] == 'ID265', 'official_name'] = "A-camallo-tetl v1"

images.loc[images['ID'] == 'ID279', 'official_name'] = "a-camallo-tetl v2"
texts.loc[texts['ID'] == 'ID279', 'official_name'] = "a-camallo-tetl v2"


In [7]:
#“Teo-xihuitl.md” in stones was already in the docs folder (REPLACE)
#Not in with pics
#ID157, 7c, plant (upper case) (explicitly says it is a plant)
#ID280, 5h, stone (lower case) (explicitly says it is a stone)
#ACTION: Capitalize official name of ID157, change to Teo-xihuitl v1, in text
#ACTION: lowercase name ID280, change to teo-xihuitl v2, in text

images.loc[images['ID'] == 'ID157', 'official_name'] = "Teo-xihuitl v1"
texts.loc[texts['ID'] == 'ID157', 'official_name'] = "Teo-xihuitl v1"

images.loc[images['ID'] == 'ID280', 'official_name'] = "teo-xihuitl v2"
texts.loc[texts['ID'] == 'ID280', 'official_name'] = "teo-xihuitl v2"

In [8]:
#“Tetlahuitl.md” in stones was already in the docs folder (REPLACE)
#Not in with pics
#ID260, 7b, plant (explicitly says it is a plant)
#ID266, many references, stone
#ACTION: Capitalize official name of ID260, change to Tetlahuitl v1, in text
#ACTION: lowercase name ID266, change to tetlahuitl v2, in text

images.loc[images['ID'] == 'ID260', 'official_name'] = "Tetlahuitl v1"
texts.loc[texts['ID'] == 'ID260', 'official_name'] = "Tetlahuitl v1"

images.loc[images['ID'] == 'ID266', 'official_name'] = "tetlahuitl v2"
texts.loc[texts['ID'] == 'ID266', 'official_name'] = "tetlahuitl v2"


# Create non-redundant df

In [9]:
import pandas as pd

# Assuming you already have `images` and `texts` as pandas DataFrames

# Ensure all spelling entries are lowercase for consistency
images["spelling"] = images["spelling"].str.lower()
texts["spelling"] = texts["spelling"].str.lower()

# Group by ID and official_name to get the text subchapters and type
text_grouped = texts.groupby(["ID", "official_name"]).agg({
    "subchapter": lambda x: "; ".join(sorted(set(x))),
    "spelling": lambda x: set(x),
    "type": lambda x: x.iloc[0]  # assumes one consistent type per ID
}).reset_index()

# Group by ID and official_name to get the illustration names and type
image_grouped = images.groupby(["ID", "official_name"]).agg({
    "illustration_name": lambda x: "; ".join(sorted(set(x))),
    "spelling": lambda x: set(x),
    "type": lambda x: x.iloc[0]  # assumes one consistent type per ID
}).reset_index()

# Merge the two grouped dataframes on ID and official_name
merged = pd.merge(text_grouped, image_grouped, on=["ID", "official_name"], how="outer", suffixes=("_text", "_image"))

# Replace NaNs with appropriate values
merged["subchapter"] = merged["subchapter"].fillna("NA")
merged["illustration_name"] = merged["illustration_name"].fillna("NA")

# Combine spellings from both sources
merged["spelling_combined"] = merged.apply(
    lambda row: (row["spelling_text"] if pd.notna(row["spelling_text"]) else set()) |
                (row["spelling_image"] if pd.notna(row["spelling_image"]) else set()),
    axis=1
)

# Determine type: prefer text type if available, else image type
merged["type"] = merged.apply(
    lambda row: row["type_text"] if pd.notna(row["type_text"]) else row["type_image"],
    axis=1
)

# Build the synonyms column
def get_synonyms(spellings, official_name):
    official_lower = official_name.lower()
    filtered = {s for s in spellings if s != official_lower}
    if not filtered:
        return "NA"
    return "; ".join(sorted(filtered))

merged["synonyms"] = merged.apply(lambda row: get_synonyms(row["spelling_combined"], row["official_name"]), axis=1)

# Select and rename the final columns
final_df = merged[["ID", "official_name", "subchapter", "illustration_name", "synonyms", "type"]].rename(columns={
    "subchapter": "text_subchapters",
    "illustration_name": "illustrations"
})

# Save to CSV
final_df.to_csv("nahuatl_names_review.csv", index=False)



# Create different sections of pages

In [10]:
# get unique IDs
image_IDs = images["ID"].unique()
text_IDs = texts["ID"].unique()

# create a sections column, to know what section each ID is
sections = []
for i in range(len(texts)):
    curr_ID = texts["ID"].iloc[i]
    if curr_ID in image_IDs:
        sections.append("plant_image")
    else:
        sections.append(texts["type"].iloc[i])
texts["section"] = sections

# Create hyperlinked text files

In [11]:
# === Paths and setup ===
directory = "./0_texts"
directory_es = "./0_texts_ES"
output_folder = "5_hyperlinked_text"
os.makedirs(output_folder, exist_ok=True)

filenames = [f for f in os.listdir(directory) if f != ".DS_Store"]

for filename in filenames:
    english_path = os.path.join(directory, filename)
    spanish_path = os.path.join(directory_es, filename)

    # Extract metadata from filename
    pg_num = filename[(filename.index("_") + 2):(filename.index("_") + 5)].lstrip('0')
    link = f"https://archive.org/details/aztec-herbal-of-1552/page/{pg_num}"
    link_md = f"[{link}]({link})  \n"

    with open(english_path, 'r') as file:
        eng_text = file.read().replace('\n', '')
        eng_text = add_space_after_punctuation(eng_text)

    with open(spanish_path, 'r', encoding='utf-8') as file:
        spa_text = file.read().replace('\n', '')
        spa_text = add_space_after_punctuation(spa_text)

    # Extract synonyms from DataFrame for this subchapter
    if filename[0] != "c":
        subchap = filename[:3].lstrip('0')
        title = f"## Subchapter {subchap}  \n"
        texts["no_zero_subchap"] = texts["subchapter"].str.lstrip("0")
        subchap_syn = texts["spelling"][texts["no_zero_subchap"] == subchap].to_list()
        subchap_off = texts["official_name"][texts["no_zero_subchap"] == subchap].to_list()
    else:
        chap = filename[7:9].lstrip('0')
        title = f"## Chapter {chap}  \n"
        subchap_syn, subchap_off = [], []

    # === Replace and format English ===
    eng_output = replace_terms(eng_text, subchap_syn, subchap_off)
    eng_bold = bold_first_sentence(eng_output)
    eng_block = (
        '=== "English :flag_us:"\n'
        f"    {eng_bold}  \n"
        f"    {link_md}"
    )

    # === Replace and format Spanish ===
    spa_output = replace_terms(spa_text, subchap_syn, subchap_off)
    spa_bold = bold_first_sentence(spa_output)
    spa_block = (
        '=== "Español :flag_mx:"\n'
        f"    {spa_bold}  \n"
    )

    # === Combine and write ===
    final_md = title + '\n' + eng_block + '\n\n' + spa_block + '\n'

    output_file = os.path.join(output_folder, f"{subchap if filename[0] != 'c' else chap}.txt")
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(final_md)

# Create subchapter and chapter markdown files

In [12]:
# get file names of all texts
directory = "./5_hyperlinked_text" 
filenames = os.listdir(directory)

# remove .DS_Store if present
if '.DS_Store' in filenames:
    filenames.remove('.DS_Store')

# create a new folder to store chapter documents
folder_name = "6_chapters"  
os.makedirs(folder_name, exist_ok=True)

# get file names of all assets
asset_dir = "./3_assets" 
asset_files = os.listdir(asset_dir)

# remove .DS_Store if present
if '.DS_Store' in asset_files:
    asset_files.remove('.DS_Store')

asset_chap_pgs = [] # save available pages with illustrations
asset_chap_inds = [] # save indices in asset files if page is available
for i in range(len(asset_files)):
    asset_id = asset_files[i][asset_files[i].index("_")+1:asset_files[i].index(".")]
    if asset_id[0]=="p":
        pg = asset_id[1:].lstrip("0")
        asset_chap_pgs.append(pg)
        asset_chap_inds.append(i)
    else:
        continue

In [13]:
for i in range(len(filenames)):
    file = filenames[i]
    id_num = file[:file.index(".")] # get chapter/subchapter number
    if not check_for_letters(id_num): # CHAPTERS
        path = directory+"/"+file
        with open(path, 'r') as file:
            text = file.read()
        path = folder_name+"/"+id_num +".md"  # change to your desired path
        with open(path, "w", encoding="utf-8") as f:
            f.write(text)
    elif check_for_letters(id_num): # SUBCHAPTERS
        path = directory+"/"+file
        with open(path, 'r') as file:
            text = file.read()
        start_ind = text.rfind("/")+1 # USE LINK TO FIND PAGE NUMBER
        end_ind = find_character_after_index(text, ")", start_ind)
        page = text[start_ind:end_ind]
        if page in asset_chap_pgs:
            ill_ind = asset_chap_inds[asset_chap_pgs.index(page)] # get index of illustration
            ill_file = asset_files[ill_ind] # get illustration file name
            ill_path = "!["+ill_file+"](assets/"+ill_file+")  " # get text to add to markdown
    
            initials = ill_file[:ill_file.index("_")] # get initials
            attributions = [] # create a list of attributions
            for i in initials:
                if i=="A":
                    attributions.append("Leaf traces by: Alejandra Rougon, UNAM ENES León, México  ")
                elif i=="D":
                    attributions.append("Leaf traces by: Dan Chitwood, Michigan State University, USA  ")
                elif i=="K":
                    attributions.append("Leaf traces by: Kylie DeViller, Acadia University, Canada  ")
                elif i=="M":
                    attributions.append("Leaf traces by: Mariana Ruíz Amaro, UNAM ENES León, México  ")
                elif i=="N":
                    attributions.append("Leaf traces by: Noé García, UNAM ENES León, México  ")
                elif i=="Z":
                    attributions.append("Leaf traces by: Zoë Migicovsky, Acadia University, Canada  ")
            attr = attributions[0] # keep to just one attribution for now

            # remove first line of text
            lines = text.splitlines(True)
            remaining_lines = lines[1:]
            text = "".join(remaining_lines)
            
            # markdown
            final_md = text + "\n" + ill_path + "\n" + attr
    
            path = folder_name+"/"+id_num+"_ill.md"  # change to your desired path, indicate illustrated
            with open(path, "w", encoding="utf-8") as f:
                f.write(final_md)
        else:
            path = folder_name+"/"+id_num+".md"  # change to your desired path, no illustration

            # remove first line of text
            lines = text.splitlines(True)
            remaining_lines = lines[1:]
            text = "".join(remaining_lines)
            
            with open(path, "w", encoding="utf-8") as f:
                f.write(text) # just text
        
        

# Plants with illustrations

In [14]:
# create a new folder to store chapter documents
folder_name = "7_plants_with_illlustrations"  
os.makedirs(folder_name, exist_ok=True)


In [15]:
# for each ID with an illustration
for id in range(len(image_IDs)):

    curr_ID = image_IDs[id] # get the ID
    
    ###
    # ILLUSTRATIONS
    ###
    image_files = np.array(images["filename"][images["ID"]==curr_ID]) #.unique() # get associated files for image
    
    # image refs
    image_refs = [] # store references to images
    for img in image_files:
        image_ref = "!["+img+"](assets/"+img+")" # generate the markdown call for the image
        image_ref = image_ref+"  "+  "\n"
        image_refs.append(image_ref)
    
    # attributions
    
    attr_each_file = [] # get attributions for each of the files
    for j in range(len(image_files)):
        initials = image_files[j].split("_")[0] # get initials for attribution
        attributions = [] # create a list of attributions
        for i in initials:
            if i=="A":
                attributions.append("Leaf traces by: Alejandra Rougon, UNAM ENES León, México  \n")
            elif i=="D":
                attributions.append("Leaf traces by: Dan Chitwood, Michigan State University, USA  \n")
            elif i=="K":
                attributions.append("Leaf traces by: Kylie DeViller, Acadia University, Canada  \n")
            elif i=="M":
                attributions.append("Leaf traces by: Mariana Ruíz Amaro, UNAM ENES León, México  \n")
            elif i=="N":
                attributions.append("Leaf traces by: Noé García, UNAM ENES León, México  \n")
            elif i=="Z":
                attributions.append("Leaf traces by: Zoë Migicovsky, Acadia University, Canada  \n")
        
        all_attr = "".join(attributions)
        attr_each_file.append(all_attr)
    
    ###
    # TEXTS
    ###
    
    # get unique subchapters for current ID
    subchapters = np.unique(texts["subchapter"][texts["ID"]==curr_ID].values)
    
    # for each subchapter, get associated hyperlinked texts
    curr_texts = []
    for s in range(len(subchapters)):
        path = "5_hyperlinked_text/"+subchapters[s].lstrip("0")+".txt"
        with open(path, 'r') as file:
            text = file.read()
        curr_texts.append(text)
    
    all_texts = "".join(curr_texts)
    
    ###
    # SYNONYMS
    ###

    # get official name
    
    if all_texts=="":
        name = images["official_name"][images["ID"]==curr_ID].unique()[0] # get official name from images if no text
    else:
        name = texts["official_name"][texts["ID"]==curr_ID].unique()[0] # get official name
    
    # Get synonyms for current ID, lowercase and deduplicate
    syns = np.unique(texts["spelling"][texts["ID"] == curr_ID].str.lower().values)
    
    # Lowercase official name for comparison
    official_name_lc = name.lower()
    
    # Remove official name from list of variants
    syns = [s for s in syns if s != official_name_lc]
    
    # Build bullet list if any variants remain
    if len(syns) > 0:
        syn_lines = ["**Variants:**\n"]
        syn_lines += [f"- {syn}" for syn in syns]
        syn_text = "\n".join(syn_lines) + "\n\n"
    else:
        syn_text = ""

    
    ###
    # PUT TOGETHER AND SAVE
    ###
    
    # put it all together
    final_text = syn_text + all_texts 
    
    for r in range(len(image_refs)):
        final_text = "".join([final_text,image_refs[r]])
        final_text = "".join([final_text,attr_each_file[r]])
    
    # save
    path = folder_name + "/" + name + ".md"
    with open(path, "w", encoding="utf-8") as f:
        f.write(final_text) # just text

generate_mkdocs_nav(folder_name, output_file=folder_name+"_nav.txt", header_name="Plants with illustrations")

Nav file written to: 7_plants_with_illlustrations_nav.txt


# Plants without illustration

In [16]:
# create a new folder to store chapter documents
folder_name = "8_plants"  
os.makedirs(folder_name, exist_ok=True)

# find IDs for plants with no images
plants = texts["ID"][texts["section"]=="plant"].unique()

# for each plant with no illustration ID
for id in range(len(plants)):

    curr_ID = plants[id] # get the ID

    ###
    # TEXTS
    ###
    
    # get unique subchapters for current ID
    subchapters = np.unique(texts["subchapter"][texts["ID"]==curr_ID].values)
    
    # for each subchapter, get associated hyperlinked texts
    curr_texts = []
    for s in range(len(subchapters)):
        path = "5_hyperlinked_text/"+subchapters[s].lstrip("0")+".txt"
        with open(path, 'r') as file:
            text = file.read()
        curr_texts.append(text)
    
    all_texts = "".join(curr_texts)
    
    ###
    # SYNONYMS
    ###
    
    # get official name
    
    name = texts["official_name"][texts["ID"]==curr_ID].unique()[0] # get official name
    
    # Get synonyms for current ID, lowercase and deduplicate
    syns = np.unique(texts["spelling"][texts["ID"] == curr_ID].str.lower().values)
    
    # Lowercase official name for comparison
    official_name_lc = name.lower()
    
    # Remove official name from list of variants
    syns = [s for s in syns if s != official_name_lc]
    
    # Build bullet list if any variants remain
    if len(syns) > 0:
        syn_lines = ["**Variants:**\n"]
        syn_lines += [f"- {syn}" for syn in syns]
        syn_text = "\n".join(syn_lines) + "\n\n"
    else:
        syn_text = ""
    
    ###
    # PUT TOGETHER AND SAVE
    ###
    
    # put it all together
    final_text = syn_text + all_texts 
    
    # save
    path = folder_name + "/" + name + ".md"
    with open(path, "w", encoding="utf-8") as f:
        f.write(final_text) # just text

generate_mkdocs_nav(folder_name, output_file=folder_name+"_nav.txt", header_name="Plants without illustrations")

Nav file written to: 8_plants_nav.txt


# Stones

In [17]:
# create a new folder to store chapter documents
folder_name = "9_stones"  
os.makedirs(folder_name, exist_ok=True)

# find IDs for stones
stones = texts["ID"][texts["section"]=="stone"].unique()

# for each stone
for id in range(len(stones)):

    curr_ID = stones[id] # get the ID

    ###
    # TEXTS
    ###
    
    # get unique subchapters for current ID
    subchapters = np.unique(texts["subchapter"][texts["ID"]==curr_ID].values)
    
    # for each subchapter, get associated hyperlinked texts
    curr_texts = []
    for s in range(len(subchapters)):
        path = "5_hyperlinked_text/"+subchapters[s].lstrip("0")+".txt"
        with open(path, 'r') as file:
            text = file.read()
        curr_texts.append(text)
    
    all_texts = "".join(curr_texts)
    
    ###
    # SYNONYMS
    ###
    
    # get official name
    
    name = texts["official_name"][texts["ID"]==curr_ID].unique()[0] # get official name
    
    # Get synonyms for current ID, lowercase and deduplicate
    syns = np.unique(texts["spelling"][texts["ID"] == curr_ID].str.lower().values)
    
    # Lowercase official name for comparison
    official_name_lc = name.lower()
    
    # Remove official name from list of variants
    syns = [s for s in syns if s != official_name_lc]
    
    # Build bullet list if any variants remain
    if len(syns) > 0:
        syn_lines = ["**Variants:**\n"]
        syn_lines += [f"- {syn}" for syn in syns]
        syn_text = "\n".join(syn_lines) + "\n\n"
    else:
        syn_text = ""
    
    ###
    # PUT TOGETHER AND SAVE
    ###
    
    # put it all together
    final_text = syn_text + all_texts 
    
    # save
    path = folder_name + "/" + name + ".md"
    with open(path, "w", encoding="utf-8") as f:
        f.write(final_text) # just text

generate_mkdocs_nav(folder_name, output_file=folder_name+"_nav.txt", header_name="Stones")

Nav file written to: 9_stones_nav.txt


# Animals

In [18]:
# create a new folder to store chapter documents
folder_name = "10_animals"  
os.makedirs(folder_name, exist_ok=True)

# find IDs for animals 
animals = texts["ID"][texts["section"]=="animal"].unique()

# for each animal
for id in range(len(animals)):

    curr_ID = animals[id] # get the ID

    ###
    # TEXTS
    ###
    
    # get unique subchapters for current ID
    subchapters = np.unique(texts["subchapter"][texts["ID"]==curr_ID].values)
    
    # for each subchapter, get associated hyperlinked texts
    curr_texts = []
    for s in range(len(subchapters)):
        path = "5_hyperlinked_text/"+subchapters[s].lstrip("0")+".txt"
        with open(path, 'r') as file:
            text = file.read()
        curr_texts.append(text)
    
    all_texts = "".join(curr_texts)
    
    ###
    # SYNONYMS
    ###
    
    # get official name
    
    name = texts["official_name"][texts["ID"]==curr_ID].unique()[0] # get official name
    
    # Get synonyms for current ID, lowercase and deduplicate
    syns = np.unique(texts["spelling"][texts["ID"] == curr_ID].str.lower().values)
    
    # Lowercase official name for comparison
    official_name_lc = name.lower()
    
    # Remove official name from list of variants
    syns = [s for s in syns if s != official_name_lc]
    
    # Build bullet list if any variants remain
    if len(syns) > 0:
        syn_lines = ["**Variants:**\n"]
        syn_lines += [f"- {syn}" for syn in syns]
        syn_text = "\n".join(syn_lines) + "\n\n"
    else:
        syn_text = ""
    
    ###
    # PUT TOGETHER AND SAVE
    ###
    
    # put it all together
    final_text = syn_text + all_texts 
    
    # save
    path = folder_name + "/" + name + ".md"
    with open(path, "w", encoding="utf-8") as f:
        f.write(final_text) # just text

generate_mkdocs_nav(folder_name, output_file=folder_name+"_nav.txt", header_name="Animals")

Nav file written to: 10_animals_nav.txt


# Birds

In [19]:
# create a new folder to store chapter documents
folder_name = "11_birds"  
os.makedirs(folder_name, exist_ok=True)

# find IDs for birds
birds = texts["ID"][texts["section"]=="bird"].unique()

# for each bird
for id in range(len(birds)):

    curr_ID = birds[id] # get the ID

    ###
    # TEXTS
    ###
    
    # get unique subchapters for current ID
    subchapters = np.unique(texts["subchapter"][texts["ID"]==curr_ID].values)
    
    # for each subchapter, get associated hyperlinked texts
    curr_texts = []
    for s in range(len(subchapters)):
        path = "5_hyperlinked_text/"+subchapters[s].lstrip("0")+".txt"
        with open(path, 'r') as file:
            text = file.read()
        curr_texts.append(text)
    
    all_texts = "".join(curr_texts)
    
    ###
    # SYNONYMS
    ###
    
    # get official name
    
    name = texts["official_name"][texts["ID"]==curr_ID].unique()[0] # get official name
    
    # Get synonyms for current ID, lowercase and deduplicate
    syns = np.unique(texts["spelling"][texts["ID"] == curr_ID].str.lower().values)
    
    # Lowercase official name for comparison
    official_name_lc = name.lower()
    
    # Remove official name from list of variants
    syns = [s for s in syns if s != official_name_lc]
    
    # Build bullet list if any variants remain
    if len(syns) > 0:
        syn_lines = ["**Variants:**\n"]
        syn_lines += [f"- {syn}" for syn in syns]
        syn_text = "\n".join(syn_lines) + "\n\n"
    else:
        syn_text = ""
    
    ###
    # PUT TOGETHER AND SAVE
    ###
    
    # put it all together
    final_text = syn_text + all_texts 
    
    # save
    path = folder_name + "/" + name + ".md"
    with open(path, "w", encoding="utf-8") as f:
        f.write(final_text) # just text

generate_mkdocs_nav(folder_name, output_file=folder_name+"_nav.txt", header_name="Birds")

Nav file written to: 11_birds_nav.txt


# Other

In [20]:
# create a new folder to store chapter documents
folder_name = "12_other"  
os.makedirs(folder_name, exist_ok=True)

# find IDs for other
other = texts["ID"][texts["section"]=="other"].unique()

# for each other
for id in range(len(other)):

    curr_ID = other[id] # get the ID

    ###
    # TEXTS
    ###
    
    # get unique subchapters for current ID
    subchapters = np.unique(texts["subchapter"][texts["ID"]==curr_ID].values)
    
    # for each subchapter, get associated hyperlinked texts
    curr_texts = []
    for s in range(len(subchapters)):
        path = "5_hyperlinked_text/"+subchapters[s].lstrip("0")+".txt"
        with open(path, 'r') as file:
            text = file.read()
        curr_texts.append(text)
    
    all_texts = "".join(curr_texts)
    
    ###
    # SYNONYMS
    ###
    
    # get official name
    
    name = texts["official_name"][texts["ID"]==curr_ID].unique()[0] # get official name
    
    # Get synonyms for current ID, lowercase and deduplicate
    syns = np.unique(texts["spelling"][texts["ID"] == curr_ID].str.lower().values)
    
    # Lowercase official name for comparison
    official_name_lc = name.lower()
    
    # Remove official name from list of variants
    syns = [s for s in syns if s != official_name_lc]
    
    # Build bullet list if any variants remain
    if len(syns) > 0:
        syn_lines = ["**Variants:**\n"]
        syn_lines += [f"- {syn}" for syn in syns]
        syn_text = "\n".join(syn_lines) + "\n\n"
    else:
        syn_text = ""
    ###
    # PUT TOGETHER AND SAVE
    ###

    # put it all together
    final_text = syn_text + all_texts 
    
    # save
    path = folder_name + "/" + name + ".md"
    with open(path, "w", encoding="utf-8") as f:
        f.write(final_text) # just text

generate_mkdocs_nav(folder_name, output_file=folder_name+"_nav.txt", header_name="Other")

Nav file written to: 12_other_nav.txt
